# Load imports 

In [22]:
## imports
import pandas as pd
import numpy as np


## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


## Load data

In [23]:
## load data on 2020 crimes in DC
## reads csv file from the web
dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")
dc_crim_2020.head()
dc_crim_2020.shape
dc_crim_2020.info()

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,BLOCK_GROUP,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID
0,-76.993031,38.956360,20162872,2020/11/14 08:28:31+00,MIDNIGHT,GUN,ASSAULT W/DANGEROUS WEAPON,735 - 939 BLOCK OF JEFFERSON STREET NE,400604.220000,143160.870000,...,009508 1,9508.0,Precinct 66,38.956352,-76.993029,NaN,2020/11/14 04:10:39+00,2020/11/14 04:15:05+00,209393235,NaN
1,-76.990031,38.839537,20162873,2020/11/14 08:31:40+00,MIDNIGHT,OTHERS,MOTOR VEHICLE THEFT,1000 - 1299 BLOCK OF MISSISSIPPI AVENUE SE,400865.649599,130192.654198,...,007304 2,7304.0,Precinct 120,38.839529,-76.990029,NaN,2020/11/14 06:47:21+00,NaN,209393236,NaN
2,-77.016457,38.910089,20162878,2020/11/14 09:00:29+00,MIDNIGHT,GUN,ASSAULT W/DANGEROUS WEAPON,1500 - 1529 BLOCK OF NEW JERSEY AVENUE NW,398572.870000,138024.530000,...,004801 3,4801.0,Precinct 18,38.910081,-77.016455,NaN,2020/11/14 07:06:46+00,NaN,209393237,NaN
3,-76.963940,38.879711,20162889,2020/11/14 10:03:15+00,MIDNIGHT,GUN,ROBBERY,700 - 799 BLOCK OF BAYLEY PLACE SE,403129.044385,134652.763610,...,007709 1,7709.0,Precinct 132,38.879703,-76.963938,NaN,2020/11/14 08:16:16+00,2020/11/14 10:03:18+00,209393238,NaN
4,-76.962445,38.854305,20162894,2020/11/14 10:27:51+00,MIDNIGHT,GUN,ROBBERY,2800 - 2899 BLOCK OF 31ST STREET SE,403259.946337,131832.667672,...,007603 4,7603.0,Precinct 113,38.854298,-76.962443,NaN,2020/11/14 08:30:06+00,2020/11/14 08:35:07+00,209393239,NaN


(27912, 25)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27912 entries, 0 to 27911
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   X                     27912 non-null  float64
 1   Y                     27912 non-null  float64
 2   CCN                   27912 non-null  int64  
 3   REPORT_DAT            27912 non-null  object 
 4   SHIFT                 27912 non-null  object 
 5   METHOD                27912 non-null  object 
 6   OFFENSE               27912 non-null  object 
 7   BLOCK                 27912 non-null  object 
 8   XBLOCK                27912 non-null  float64
 9   YBLOCK                27912 non-null  float64
 10  WARD                  27912 non-null  int64  
 11  ANC                   27912 non-null  object 
 12  DISTRICT              27883 non-null  float64
 13  PSA                   27877 non-null  float64
 14  NEIGHBORHOOD_CLUSTER  27912 non-null  object 
 15  BLOCK_GROUP        

## 1. Aggregation

### Example of grouping by one variable and doing one aggregation 

**Task**: find and print the number of unique offense types (`OFFENSE`) by ward (`WARD`)

In [24]:
## way one: use built in unique function
## documentation here: https://pandas.pydata.org/docs/reference/api/pandas.Series.nunique.html
unique_off_byward = dc_crim_2020.groupby('WARD').agg({'OFFENSE': 
                                'nunique'}).reset_index()
unique_off_byward

## way two: use lambda function
## documentation here: https://www.w3schools.com/python/python_lambda.asp
unique_off_byward_v2 = dc_crim_2020.groupby('WARD').agg({'OFFENSE': 
                    lambda x: len(x.unique())}).reset_index()
unique_off_byward_v2

,WARD,OFFENSE
0,1,9
1,2,9
2,3,8
3,4,8
4,5,9
5,6,9
6,7,9
7,8,9


,WARD,OFFENSE
0,1,9
1,2,9
2,3,8
3,4,8
4,5,9
5,6,9
6,7,9
7,8,9


### Example of grouping by one variable and providing two summaries of the same variable

**Task**: previous task showed number of offenses by ward but want to find out content of offenses in each ward; create an aggregation that summarizes both the number of unique offenses by ward and what those offenses are -- for instance, by pasting the unique offenses in that ward separated by the ";" (e.g., Theft; Burglary;...)

*Hint*: you can use the join command to paste together a list separated by some delimiter. 
    - The syntax, if we are using the comma delimiter, is: ", ".join(nameoflist)

In [25]:
## one way to solve -- write out the whole function inside the agg and use default varnames
unique_off_valuecount_byward = dc_crim_2020.groupby('WARD').agg({'OFFENSE': ['nunique',
                    lambda x: "; ".join(sorted(x.unique()))]}).reset_index()


In [26]:

## a second way to solve --- write the function outside of the agg and then feed that function
## to the agg (still uses lambda)
def find_join_unique(x):
    
    sorted_un = sorted(x.unique())
    joined_un = "; ".join(sorted_un)
    return(joined_un)

unique_off_valuecount_byward_v2 = dc_crim_2020.groupby('WARD').agg({'OFFENSE': ['nunique',
                                            lambda x: find_join_unique(x)]}).reset_index()


## a third way to solve --- give more informative column names by using a diff structure
## inside agg
unique_off_valuecount_byward_bettername = dc_crim_2020.groupby("WARD").agg(
                                            count_offense =
                                            ('OFFENSE', lambda x: 
                                            len(x.unique())), 
                                            name_offenses =
                                            ('OFFENSE', lambda x: 
                                        find_join_unique(x))).reset_index()

unique_off_valuecount_byward_bettername

,WARD,count_offense,name_offenses
0,1,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
1,2,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
2,3,8,ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
3,4,8,ASSAULT W/DANGEROUS WEAPON; BURGLARY; HOMICIDE...
4,5,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
5,6,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
6,7,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...
7,8,9,ARSON; ASSAULT W/DANGEROUS WEAPON; BURGLARY; H...


### Example of grouping by two variables 

**Task**: group by ward (`WARD`) and police shift (`SHIFT`) and find the offense that is most common in that ward and shift

In [27]:
## way one- flexible enough to pull either most or least frequent cat
top_w_shift = dc_crim_2020.groupby(['WARD', 'SHIFT']).agg({'OFFENSE': 
                            lambda x: 
                                x.value_counts(sort = True, 
                        ascending = False).index[0]}).reset_index()
top_w_shift

,WARD,SHIFT,OFFENSE
0,1,DAY,THEFT/OTHER
1,1,EVENING,THEFT/OTHER
2,1,MIDNIGHT,THEFT/OTHER
3,2,DAY,THEFT/OTHER
4,2,EVENING,THEFT/OTHER
5,2,MIDNIGHT,THEFT/OTHER
6,3,DAY,THEFT/OTHER
7,3,EVENING,THEFT/OTHER
8,3,MIDNIGHT,THEFT/OTHER
9,4,DAY,THEFT F/AUTO


In [28]:
## way two: function defined outside the pandas dataframe (preview for next session)

def most_common(one_col: pd.Series):
    
    ## sort values
    sorted_series = one_col.value_counts(sort = True, ascending = False)
    
    ## get top 
    top = sorted_series.index[0]
    
    ## return
    return(top)

In [29]:
top_w_shift_alternate = dc_crim_2020.groupby(['WARD', 'SHIFT']).agg({'OFFENSE': 
                                                lambda x: most_common(x)}).reset_index()

In [30]:
top_w_shift_alternate

,WARD,SHIFT,OFFENSE
0,1,DAY,THEFT/OTHER
1,1,EVENING,THEFT/OTHER
2,1,MIDNIGHT,THEFT/OTHER
3,2,DAY,THEFT/OTHER
4,2,EVENING,THEFT/OTHER
5,2,MIDNIGHT,THEFT/OTHER
6,3,DAY,THEFT/OTHER
7,3,EVENING,THEFT/OTHER
8,3,MIDNIGHT,THEFT/OTHER
9,4,DAY,THEFT F/AUTO


In [31]:
## way three to solve: use the built-in mode command

dc_crim_2020.groupby(['WARD', 'SHIFT']).agg({'OFFENSE': 
                                    lambda x: x.mode()})


OFFENSE
WARD SHIFT                               
1    DAY                      THEFT/OTHER
     EVENING                  THEFT/OTHER
     MIDNIGHT                 THEFT/OTHER
2    DAY                      THEFT/OTHER
     EVENING                  THEFT/OTHER
     MIDNIGHT                 THEFT/OTHER
3    DAY                      THEFT/OTHER
     EVENING                  THEFT/OTHER
     MIDNIGHT                 THEFT/OTHER
4    DAY                     THEFT F/AUTO
     EVENING                  THEFT/OTHER
     MIDNIGHT                 THEFT/OTHER
5    DAY                     THEFT F/AUTO
     EVENING                  THEFT/OTHER
     MIDNIGHT                 THEFT/OTHER
6    DAY                      THEFT/OTHER
     EVENING                  THEFT/OTHER
     MIDNIGHT                 THEFT/OTHER
7    DAY                     THEFT F/AUTO
     EVENING                  THEFT/OTHER
     MIDNIGHT                 THEFT/OTHER
8    DAY                     THEFT F/AUTO
     EVENING                  THEFT/OTHER
     MIDNIGHT  ASSAULT W/DANGEROUS WEAPON

## Summarizing over all rows or all columns (without grouping)

We can also use the `apply` function to summarize rows or columns efficiently

**Task**: find the mean lat and longitude of crimes using line of code

In [32]:
## pandas built-in
dc_crim_2020[["LATITUDE", "LONGITUDE"]].apply('mean', axis = 0)

## numpy
dc_crim_2020[["LATITUDE", "LONGITUDE"]].apply(np.mean, axis = 0)


LATITUDE     38.907620
LONGITUDE   -77.007041
dtype: float64

LATITUDE     38.907620
LONGITUDE   -77.007041
dtype: float64

In [33]:
## what happens if we change the axis parameter from
## axis = 0 to axis = 1?

dc_crim_2020[["LATITUDE", "LONGITUDE"]].apply(np.mean, axis = 1) # just taking mean across 
# rows which isn't informative 

0       -19.018339
1       -19.075250
2       -19.053187
3       -19.042118
4       -19.054073
           ...    
27907   -19.053069
27908   -19.039014
27909   -19.022376
27910   -19.058229
27911   -19.028111
Length: 27912, dtype: float64

## 2. Creating new columns/transforming their type 

### Simple np.where

`np.where` is in the numpy package (aliased as `np`) and operates similar to `ifelse` in R

**Task**: create an indicator `is_theft` for any offense that contains the word "THEFT"
    
**Task**: create an indicator `is_theft_notmotor` for any offense that contains the word "THEFT" but does not contain the word "MOTOR"

In [34]:

## approach 1: np.where 
dc_crim_2020['is_theft'] = np.where(dc_crim_2020.OFFENSE.str.contains("THEFT"), True, False)

## approach 2: list iteration 
## returning True if theft is in
## offense, false otherwise
## note that offense is an arbitrary placeholder
## and could be replaced with x, o, etc.
dc_crim_2020['is_theft_alternate'] = [True if "THEFT" in offense
                                      else False
                                      for offense in dc_crim_2020.OFFENSE]

pd.crosstab(dc_crim_2020.is_theft, dc_crim_2020.OFFENSE)
pd.crosstab(dc_crim_2020.is_theft, dc_crim_2020.is_theft_alternate)

## approach three: skip the np.where and just create a boolean
dc_crim_2020['is_theft_alternate2'] = dc_crim_2020.OFFENSE.str.contains("THEFT")



## then, create variable for whether it is a non-motor theft
dc_crim_2020['is_theft_notmotor'] = np.where(dc_crim_2020.OFFENSE.str.contains("THEFT") & 
                                            ~dc_crim_2020.OFFENSE.str.contains("MOTOR"), 
                                             True, False)


pd.crosstab(dc_crim_2020.is_theft_notmotor, dc_crim_2020.OFFENSE)

OFFENSE,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER
is_theft,,,,,,,,,
False,13,1630,1442,196,0,1997,178,0,0
True,0,0,0,0,3257,0,0,8275,10924


is_theft_alternate,False,True
is_theft,,
False,5456,0
True,0,22456


OFFENSE,ARSON,ASSAULT W/DANGEROUS WEAPON,BURGLARY,HOMICIDE,MOTOR VEHICLE THEFT,ROBBERY,SEX ABUSE,THEFT F/AUTO,THEFT/OTHER
is_theft_notmotor,,,,,,,,,
False,13,1630,1442,196,3257,1997,178,0,0
True,0,0,0,0,0,0,0,8275,10924


### np.select

**Task**: create a new variable, `offense_summary`, where you:
        
- Recode theft offenses that use a gun or knife as the method (`METHOD`) as: violent theft
- Recode non-theft offenses that use a gun or knife as the method as: violent other
- Recode all other as non-violent 

In [35]:
## for convenience, copy to shorter name
df = dc_crim_2020.copy()

## step 1: create conditions
conditions_offense = [df['is_theft'] & df['METHOD'].isin(['GUN', 'KNIFE']),
                     ~df['is_theft'] & df['METHOD'].isin(['GUN', 'KNIFE'])]

## step 2: create what each condition resolves/codes to
code_to = ['violent theft', 'violent other']

## step 3: feed np.select the list of 
## conditions and what they code to
df['offense_summary'] = np.select(conditions_offense,
                                 code_to, 
                                 default = 'non-violent')

## double check outputs
df.offense_summary.value_counts()

## check
df.loc[(df.is_theft) &
  (df.METHOD.isin(['GUN', 'KNIFE'])),
['OFFENSE', 'offense_summary', 'METHOD']].head()


non-violent      25239
violent other     2638
violent theft       35
Name: offense_summary, dtype: int64

,OFFENSE,offense_summary,METHOD
1046,THEFT/OTHER,violent theft,KNIFE
1352,THEFT/OTHER,violent theft,KNIFE
2017,THEFT/OTHER,violent theft,KNIFE
2319,THEFT/OTHER,violent theft,KNIFE
2799,THEFT/OTHER,violent theft,KNIFE


### Using built-in pandas methods

Another way of creating variables, used either alone or in combination with np.where and np.select is to use pandas built in `str` methods

Basic structure is: `df['namestringcol'].str.someoperation`

**Task**: using a `str` method, create a new variable--`OFFENSE_NOSP`--that replaces spaces in the `OFFENSE` column with underscores

In [36]:
dc_crim_2020['OFFENSE_NOSP'] = dc_crim_2020.OFFENSE.str.replace(" ", "_")

## print example of random sample of ones that should
## be changed; detect this via str.contains
dc_crim_2020.loc[dc_crim_2020.OFFENSE.str.contains(" "),
            ["OFFENSE", "OFFENSE_NOSP"]].sample(n = 10)

,OFFENSE,OFFENSE_NOSP
16407,THEFT F/AUTO,THEFT_F/AUTO
12472,MOTOR VEHICLE THEFT,MOTOR_VEHICLE_THEFT
15273,ASSAULT W/DANGEROUS WEAPON,ASSAULT_W/DANGEROUS_WEAPON
3066,THEFT F/AUTO,THEFT_F/AUTO
341,THEFT F/AUTO,THEFT_F/AUTO
20647,THEFT F/AUTO,THEFT_F/AUTO
24617,ASSAULT W/DANGEROUS WEAPON,ASSAULT_W/DANGEROUS_WEAPON
13148,THEFT F/AUTO,THEFT_F/AUTO
23856,THEFT F/AUTO,THEFT_F/AUTO
14519,THEFT F/AUTO,THEFT_F/AUTO


### Transforming column types

**Task**: check the type of the `START_DATE` column

In [37]:
dc_crim_2020.START_DATE.dtype

dtype('O')

**Task**: recast the `START_DATE` column to datetime, calling this `START_DATE_CLEAN`; coerce errors rather than cleaning the string; check the type of that new col

In [38]:
dc_crim_2020["START_DATE_CLEAN"] = pd.to_datetime(dc_crim_2020.START_DATE, 
                                                 errors = "coerce")

dc_crim_2020.START_DATE_CLEAN.dtype

datetime64[ns, UTC]

**Task:** print the min and max of `START_DATE_CLEAN`. What happens if you try to do this with `START_DATE`?

In [39]:
dc_crim_2020.START_DATE_CLEAN.min()
dc_crim_2020.START_DATE_CLEAN.max()


### trying with START_DATE
### uncomment to see error
# dc_crim_2020.START_DATE.min()

Timestamp('1940-02-17 00:00:16+0000', tz='UTC')

Timestamp('2021-01-01 02:30:21+0000', tz='UTC')

## 3. Row and column subsetting 

### Row subsetting

**Task**: filter to crime reports about theft using the `is_theft` definition and that are in ward 3

In [40]:
theft_w3 = dc_crim_2020[(dc_crim_2020.is_theft) &
                       (dc_crim_2020.WARD == 3)].copy()

## one way to check that the row filtering worked: assert
## documentation here: https://python-reference.readthedocs.io/en/latest/docs/statements/assert.html
assert all(theft_w3.is_theft == True)
assert all(theft_w3.WARD == 3)

## another way to check: value counts
theft_w3.is_theft.value_counts()
theft_w3.WARD.value_counts()

True    1521
Name: is_theft, dtype: int64

3    1521
Name: WARD, dtype: int64

### Column subsetting


**Task**: select two columns--`START_DATE` and `END_DATE`--and print the head of the dataframe

In [41]:
dc_crim_2020[['START_DATE', 'END_DATE']].head()

,START_DATE,END_DATE
0,2020/11/14 04:10:39+00,2020/11/14 04:15:05+00
1,2020/11/14 06:47:21+00,NaN
2,2020/11/14 07:06:46+00,NaN
3,2020/11/14 08:16:16+00,2020/11/14 10:03:18+00
4,2020/11/14 08:30:06+00,2020/11/14 08:35:07+00


**Task**: use list comprehension to automate this a bit and select all columns with the word "DATE" in the name; print the head of the dataframe

In [42]:
dc_crim_2020[[col for col in dc_crim_2020.columns if 
             "DATE" in col]].head()

,START_DATE,END_DATE,START_DATE_CLEAN
0,2020/11/14 04:10:39+00,2020/11/14 04:15:05+00,2020-11-14 04:10:39+00:00
1,2020/11/14 06:47:21+00,NaN,2020-11-14 06:47:21+00:00
2,2020/11/14 07:06:46+00,NaN,2020-11-14 07:06:46+00:00
3,2020/11/14 08:16:16+00,2020/11/14 10:03:18+00,2020-11-14 08:16:16+00:00
4,2020/11/14 08:30:06+00,2020/11/14 08:35:07+00,2020-11-14 08:30:06+00:00


**Task**: filter reports of crime in `START_DATE_CLEAN` before May 2020 and that are located in ward 1; select the ward and `START_DATE` column and print a random sample of rows

In [43]:
dc_crim_2020.loc[(dc_crim_2020.START_DATE_CLEAN < "2020/05/01") &
                (dc_crim_2020.WARD == 1),
                ["START_DATE", "WARD"]].sample(n = 10)


,START_DATE,WARD
2587,2020/02/03 05:06:04+00,1
3073,2020/02/27 20:09:58+00,1
27448,2020/04/24 20:19:20+00,1
5783,2020/02/06 20:39:53+00,1
20896,2020/02/08 20:30:27+00,1
7545,2020/04/20 16:42:48+00,1
6246,2020/02/23 14:00:22+00,1
20367,2020/01/30 20:30:57+00,1
19971,2020/03/07 00:47:19+00,1
19156,2020/02/12 22:00:21+00,1


### An additional way: map.recode

**Task**: recode shifts that are MIDNIGHT or EVENING as "nighttime"; code other shift to daytime

In [44]:
shifts_dict = {'MIDNIGHT': 'nighttime',
              'EVENING': 'nighttime'}

dc_crim_2020['summary_shifts'] = dc_crim_2020.SHIFT.map(shifts_dict).fillna("daytime")

pd.crosstab(dc_crim_2020.summary_shifts,
           dc_crim_2020.SHIFT)

SHIFT,DAY,EVENING,MIDNIGHT
summary_shifts,,,
daytime,10091,0,0
nighttime,0,12312,5509
